<a href="https://colab.research.google.com/github/sj123r/Machine-Learning/blob/main/%EA%B0%9D%EC%B2%B4%EC%A7%80%ED%96%A5_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class Perceptron(object):
  def __init__(self, eta=0.01, n_iter=50, random_state=1):
    #퍼셉트론 초기화 과정. 학습률, 편향, epoch이 해당.
    self.eta = eta
    self.n_iter = n_iter
    self.random_state = random_state

  def fit(self, X, y):
    rgen = np.random.RandomState(self.random_state)                 #RandomState를 고정해서 fit 함수를 돌릴 때마다 생성된 가중치가 같도록.
    self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1]) #가우시안 distribution을 통한 랜덤한 가중치들 생성.
    self.errors_ = []

    for _ in range(self.n_iter):
      errors = 0
      for xi, target in zip(X, y):
        update = self.eta * (target - self.predict(xi))
        self.w_[1:] += update * xi
        self.w_[0] += update
        errors += int(update != 0.0)
      self.errors_.append(errors)
    
    return self

  def net_input(self, X):
    return np.dot(X, self.w_[1:]) + self.w_[0]
  
  def predict(self, X):
    return np.where(self.net_input(X) >= 0.0, 1, -1)